In [63]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [64]:
from skforecast.drift_detection import RangeDriftDetector
from pprint import pprint
from copy import deepcopy

In [65]:
y = pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D'), name="y")
series_df = pd.DataFrame(
    {
        'series_1': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
        'series_2': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
    }
)
series_df_long = series_df.stack().to_frame().rename(columns={0: 'y'}).swaplevel(0,1).sort_index()
exog = pd.Series(np.random.rand(5000), index=y.index, name="temp")
exog_df = pd.DataFrame(index=y.index)
for i in range(5):
    exog_df[f"feature_{i}"] = np.random.rand(5000)
for i in range(5):
    exog_df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)


series_dict = series_df.to_dict(orient='series')
exog_dict = {'series_1': exog_df, 'series_2': exog_df}
exog_df_long = [
    exog_df.copy().assign(series_id='series_1'),
    exog_df.copy().assign(series_id='series_2')
]
exog_df_long = pd.concat(exog_df_long).set_index('series_id', append=True).reorder_levels(['series_id', None]).sort_index()

# FIT

In [66]:
range_detector = RangeDriftDetector()
range_detector.fit(y)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'y': (np.float64(0.000380756499260837), np.float64(0.9998805232747526))}
['y']


In [67]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0002008168090453788),
              np.float64(0.9999843897950614)),
 'series_2': (np.float64(0.00021221547443395217),
              np.float64(0.9997996332760751))}
['series_1', 'series_2']


In [68]:
range_detector = RangeDriftDetector()
range_detector.fit(series_dict)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0002008168090453788),
              np.float64(0.9999843897950614)),
 'series_2': (np.float64(0.00021221547443395217),
              np.float64(0.9997996332760751))}
['series_1', 'series_2']


In [69]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df_long)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0002008168090453788),
              np.float64(0.9999843897950614)),
 'series_2': (np.float64(0.00021221547443395217),
              np.float64(0.9997996332760751))}
['series_1', 'series_2']


In [70]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'temp': (np.float64(6.61205975738266e-06), np.float64(0.9997945077825358))}
['temp']


In [71]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'cat_feature_0': {'C', 'B', 'A'},
 'cat_feature_1': {'C', 'B', 'A'},
 'cat_feature_2': {'C', 'B', 'A'},
 'cat_feature_3': {'C', 'B', 'A'},
 'cat_feature_4': {'C', 'B', 'A'},
 'feature_0': (np.float64(0.0001984849088711993),
               np.float64(0.9998052878019418)),
 'feature_1': (np.float64(1.583729243970211e-05),
               np.float64(0.9997643665826184)),
 'feature_2': (np.float64(2.4902578088181748e-05),
               np.float64(0.9994669062686304)),
 'feature_3': (np.float64(0.00014640894253425962),
               np.float64(0.9999476855623223)),
 'feature_4': (np.float64(0.00011041733054084624),
               np.float64(0.9996506660506206))}
['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4']


In [72]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_dict)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'C', 'B', 'A'},
              'cat_feature_2': {'C', 'B', 'A'},
              'cat_feature_3': {'C', 'B', 'A'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(0.0001984849088711993),
                            np.float64(0.9998052878019418)),
              'feature_1': (np.float64(1.583729243970211e-05),
                            np.float64(0.9997643665826184)),
              'feature_2': (np.float64(2.4902578088181748e-05),
                            np.float64(0.9994669062686304)),
              'feature_3': (np.float64(0.00014640894253425962),
                            np.float64(0.9999476855623223)),
              'feature_4': (np.float64(0.00011041733054084624),
                            np.float64(0.9996506660506206))},
 'series_2': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'C', 'B', 'A'},
              'cat_feature_2': {'C', 'B'

In [73]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df_long)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'C', 'B', 'A'},
              'cat_feature_2': {'C', 'B', 'A'},
              'cat_feature_3': {'C', 'B', 'A'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(0.0001984849088711993),
                            np.float64(0.9998052878019418)),
              'feature_1': (np.float64(1.583729243970211e-05),
                            np.float64(0.9997643665826184)),
              'feature_2': (np.float64(2.4902578088181748e-05),
                            np.float64(0.9994669062686304)),
              'feature_3': (np.float64(0.00014640894253425962),
                            np.float64(0.9999476855623223)),
              'feature_4': (np.float64(0.00011041733054084624),
                            np.float64(0.9996506660506206))},
 'series_2': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'C', 'B', 'A'},
              'cat_feature_2': {'C', 'B'

# Predict

In [74]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.series_values_range_)
pprint(range_detector.exog_values_range_)


{'y': (np.float64(0.000380756499260837), np.float64(0.9998805232747526))}
{'temp': (np.float64(6.61205975738266e-06), np.float64(0.9997945077825358))}


In [84]:
last_window_valid = y.tail(100).copy()
last_window_invalid = y.tail(100).copy()
last_window_invalid.iloc[5] = 10_000    
#range_detector.predict(last_window=last_window_valid, verbose=True, suppress_warnings=True)
range_detector.predict(last_window=last_window_invalid, verbose=False, suppress_warnings=False)
range_detector


╭──────────────────────────────── UnknownLevelWarning ─────────────────────────────────╮
│ 'y' was not seen during training. Its range is unknown.                              │
│                                                                                      │
│ Category : skforecast.exceptions.UnknownLevelWarning                                 │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :433                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=UnknownLevelWarning)             │
╰──────────────────────────────────────────────────────────────────────────────────────╯

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0002008168090453788), np.float64(0.9999843897950614)), 'series_2': (np.float64(0.00021221547443395217), np.float64(0.9997996332760751))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.0001984849088711993), np.float64(0.9998052878019418)), 'feature_1': (np.float64(1.583729243970211e-05), np.float64(0.9997643665826184)), 'feature_2': (np.float64(2.4902578088181748e-05), np.float64(0.9994669062686304)), 'feature_3': (np.float64(0.00014640894253425962), np.float64(0.9999476855623223)), 'feature_4': (np.float64(0.00011041733054084624), np.float64(0.9996506660506206)), 'cat_feature_0': {'C', 'B', 'A'}, 'cat_feature_1': {'C', 'B', 'A'}, 'cat_feature_2': {'C', 'B', 'A'}, 'cat_feature_3': {'C', 'B', 'A'}, 'cat_feature_4': {'C', 'B', 'A'}}, 'series_2': {'feature_0': (np.float64(0.0001984849088711993), np.float64(0.9998052878019418)), 'feature_1': (np.float64(1.583729243970211e-05), np.float64(0.999764366582618

In [76]:
range_detector.fit(series_df)
last_window_valid_df = series_df.tail(100).copy()
last_window_invalid_df = series_df.tail(100).copy()
last_window_invalid_df.iloc[1, [0, 1]] = 10_000
range_detector.predict(last_window=last_window_valid_df)
range_detector.predict(last_window=last_window_invalid_df)
range_detector

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has one or more values outside the range seen during training [0.00020,   │
│ 0.99998]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has one or more values outside the range seen during training [0.00021,   │
│ 0.99980]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── Out-of-range summary ──────────────────────────────╮
│ Series:                                                                          │
│ 'series_1' has one or more values outside the observed range [0.00020, 0.99998]. │
│ 'series_2' has one or more values outside the observed range [0.00021, 0.99980]. │
│                                                                                  │
│ Exogenous Variables:                                                             │
│ No exogenous variables with out-of-range values found.                           │
│                                                                                  │
│                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────╯

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0002008168090453788), np.float64(0.9999843897950614)), 'series_2': (np.float64(0.00021221547443395217), np.float64(0.9997996332760751))} 
Exogenous value ranges: None 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: None 

In [77]:
range_detector.fit(series_dict)
last_window_valid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
last_window_invalid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
for key in last_window_invalid_dict.keys():
    last_window_invalid_dict[key].iloc[1] = 10_000
range_detector.predict(last_window=last_window_valid_dict)
range_detector.predict(last_window=last_window_invalid_dict)
range_detector

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── Out-of-range summary ──────────────────────────────╮
│ Series:                                                                          │
│ 'series_1' has one or more values outside the observed range [0.00020, 0.99998]. │
│ 'series_2' has one or more values outside the observed range [0.00021, 0.99980]. │
│                                                                                  │
│ Exogenous Variables:                                                             │
│ No exogenous variables with out-of-range values found.                           │
│                                                                                  │
│                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────╯

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0002008168090453788), np.float64(0.9999843897950614)), 'series_2': (np.float64(0.00021221547443395217), np.float64(0.9997996332760751))} 
Exogenous value ranges: None 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: None 

In [78]:
range_detector.fit(series_df_long)
last_window_valid_df_long= series_df_long.groupby(level=0).tail(100).copy()
last_window_invalid_df_long = series_df_long.groupby(level=0).tail(100).copy()
for key in last_window_invalid_df_long.index.get_level_values(0).unique():
    last_window_invalid_df_long.loc[key, pd.Timestamp("2013-06-02")] = 10_000
range_detector.predict(last_window=last_window_valid_df_long)
range_detector.predict(last_window=last_window_invalid_df_long)

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has one or more values outside the range seen during training [0.00020,   │
│ 0.99998]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has one or more values outside the range seen during training [0.00021,   │
│ 0.99980]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── Out-of-range summary ──────────────────────────────╮
│ Series:                                                                          │
│ 'series_1' has one or more values outside the observed range [0.00020, 0.99998]. │
│ 'series_2' has one or more values outside the observed range [0.00021, 0.99980]. │
│                                                                                  │
│ Exogenous Variables:                                                             │
│ No exogenous variables with out-of-range values found.                           │
│                                                                                  │
│                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────╯

(True, ['series_1', 'series_2'], [])

In [79]:
range_detector.fit(series_df, exog=exog)
print(range_detector)
exog_valid = exog.tail(100).copy()
exog_invalid = exog.tail(100).copy()
exog_invalid.iloc[5] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0002008168090453788), np.float64(0.9999843897950614)), 'series_2': (np.float64(0.00021221547443395217), np.float64(0.9997996332760751))} 
Exogenous value ranges: {'temp': (np.float64(6.61205975738266e-06), np.float64(0.9997945077825358))} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: ['temp'] 



╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'temp' has one or more values outside the range seen during training [0.00001,       │
│ 0.99979]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────── Out-of-range summary ───────────────────────────╮
│ Series:                                                                    │
│ No series with out-of-range values found.                                  │
│                                                                            │
│ Exogenous Variables:                                                       │
│ 'temp': 'temp' has one or more values outside the observed range [0.00001, │
│ 0.99979].                                                                  │
│                                                                            │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

(True, [], ['temp'])

In [80]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has one or more values outside the range seen during training [0.00020,  │
│ 0.99981]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has one or more values outside the range seen during training [0.00002,  │
│ 0.99976]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────── Out-of-range summary ───────────────────────────╮
│ Series:                                                                    │
│ No series with out-of-range values found.                                  │
│                                                                            │
│ Exogenous Variables:                                                       │
│ 'feature_0': 'feature_0' has one or more values outside the observed range │
│ [0.00020, 0.99981].                                                        │
│ 'feature_1': 'feature_1' has one or more values outside the observed range │
│ [0.00002, 0.99976].                                                        │
│                                                                            │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

(True, [], ['feature_0', 'feature_1'])

In [81]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has one or more values outside the range seen during training [0.00020,  │
│ 0.99981]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has one or more values outside the range seen during training [0.00002,  │
│ 0.99976]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────── Out-of-range summary ───────────────────────────╮
│ Series:                                                                    │
│ No series with out-of-range values found.                                  │
│                                                                            │
│ Exogenous Variables:                                                       │
│ 'feature_0': 'feature_0' has one or more values outside the observed range │
│ [0.00020, 0.99981].                                                        │
│ 'feature_1': 'feature_1' has one or more values outside the observed range │
│ [0.00002, 0.99976].                                                        │
│                                                                            │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

(True, [], ['feature_0', 'feature_1'])

In [82]:
range_detector = RangeDriftDetector()

range_detector.fit(series=series_dict, exog=exog_dict)
print(range_detector)
exog_valid = deepcopy(exog_dict)
exog_invalid = deepcopy(exog_dict)
for key in exog_invalid.keys():
    
    exog_invalid[key].iloc[1, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0002008168090453788), np.float64(0.9999843897950614)), 'series_2': (np.float64(0.00021221547443395217), np.float64(0.9997996332760751))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.0001984849088711993), np.float64(0.9998052878019418)), 'feature_1': (np.float64(1.583729243970211e-05), np.float64(0.9997643665826184)), 'feature_2': (np.float64(2.4902578088181748e-05), np.float64(0.9994669062686304)), 'feature_3': (np.float64(0.00014640894253425962), np.float64(0.9999476855623223)), 'feature_4': (np.float64(0.00011041733054084624), np.float64(0.9996506660506206)), 'cat_feature_0': {'C', 'B', 'A'}, 'cat_feature_1': {'C', 'B', 'A'}, 'cat_feature_2': {'C', 'B', 'A'}, 'cat_feature_3': {'C', 'B', 'A'}, 'cat_feature_4': {'C', 'B', 'A'}}, 'series_2': {'feature_0': (np.float64(0.0001984849088711993), np.float64(0.9998052878019418)), 'feature_1': (np.float64(1.583729243970211e-05), np.float64(0.999764366582618

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00020, 0.99981]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00002, 0.99976]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00020, 0.99981]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00002, 0.99976]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :187                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Out-of-range summary ───────────────────────────╮
│ Series:                                                                   │
│ No series with out-of-range values found.                                 │
│                                                                           │
│ Exogenous Variables:                                                      │
│ 'series_1': 'feature_0' has one or more values outside the observed range │
│ [0.00020, 0.99981].                                                       │
│ 'series_1': 'feature_1' has one or more values outside the observed range │
│ [0.00002, 0.99976].                                                       │
│ 'series_2': 'feature_0' has one or more values outside the observed range │
│ [0.00020, 0.99981].                                                       │
│ 'series_2': 'feature_1' has one or more values outside the observed range │
│ [0.00002, 0.99976].                                                       │
│                                                                           │
│                                                                           │
╰───────────────────────────────────────────────────────────────────────────╯

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0002008168090453788), np.float64(0.9999843897950614)), 'series_2': (np.float64(0.00021221547443395217), np.float64(0.9997996332760751))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.0001984849088711993), np.float64(0.9998052878019418)), 'feature_1': (np.float64(1.583729243970211e-05), np.float64(0.9997643665826184)), 'feature_2': (np.float64(2.4902578088181748e-05), np.float64(0.9994669062686304)), 'feature_3': (np.float64(0.00014640894253425962), np.float64(0.9999476855623223)), 'feature_4': (np.float64(0.00011041733054084624), np.float64(0.9996506660506206)), 'cat_feature_0': {'C', 'B', 'A'}, 'cat_feature_1': {'C', 'B', 'A'}, 'cat_feature_2': {'C', 'B', 'A'}, 'cat_feature_3': {'C', 'B', 'A'}, 'cat_feature_4': {'C', 'B', 'A'}}, 'series_2': {'feature_0': (np.float64(0.0001984849088711993), np.float64(0.9998052878019418)), 'feature_1': (np.float64(1.583729243970211e-05), np.float64(0.999764366582618